# <font color="#9ACD32">Movie Recommendation Model</font>

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Data loading
movies_list  = pd.read_csv('movies.csv')

In [3]:
# Viewing first few columns of the DataFrame(movies_list)
movies_list.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


## DATA ANALYSIS

In [4]:
# determinig the length of dataframe
len(movies_list)

4803

In [5]:
# determining the shape of dataframe
display(movies_list.shape)

(4803, 24)

In [6]:
# determining the datatypes of columns 
display(movies_list.dtypes)

index                     int64
budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
cast                     object
crew                     object
director                 object
dtype: object

In [7]:
# Summary of the DataFrame(movies_list)
display(movies_list.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

In [8]:
# Checking if there is any missing values in the DataFrame
display(movies_list.isna().sum())

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

## RELEVANT COLUMNS FOR THE RECOMMENDATION SYSTEM

In [9]:
selected_features = ['genres', 'tagline', 'keywords', 'overview', 'cast', 'director']
print(selected_features)

['genres', 'tagline', 'keywords', 'overview', 'cast', 'director']


In [10]:
# filling the missing values of selected feautres with null string
movies_list[selected_features] = movies_list[selected_features].fillna('')

In [11]:
movies_list.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [12]:
# rechecking if there is any missing values in selected features
display (movies_list[selected_features].isna().sum())

genres      0
tagline     0
keywords    0
overview    0
cast        0
director    0
dtype: int64

In [13]:
# concatenating the selected features
def combine_features(row):
    return row['genres'] + ' ' + row['overview'] + ' ' + row['cast'] + ' ' + row['director'] + ' ' + row['tagline'] + ' ' + row['keywords']

movies_list['combined_features'] = movies_list.apply(combine_features, axis=1)

In [14]:
print(movies_list['combined_features'].head())

0    Action Adventure Fantasy Science Fiction In th...
1    Adventure Fantasy Action Captain Barbossa, lon...
2    Action Adventure Crime A cryptic message from ...
3    Action Crime Drama Thriller Following the deat...
4    Action Adventure Science Fiction John Carter i...
Name: combined_features, dtype: object


In [15]:
print(movies_list['combined_features'].isna().sum())

0


## Converting the Text Data to Feature Vectors

In [16]:
vectorizer = TfidfVectorizer(stop_words='english') 
feature_vector = vectorizer.fit_transform(movies_list['combined_features'])

In [17]:
display (feature_vector.shape)
display (pd.DataFrame(feature_vector.toarray()))

(4803, 30308)

,0,1,2,3,4,5,6,7,8,9,...,30298,30299,30300,30301,30302,30303,30304,30305,30306,30307
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
print(feature_vector)

  (0, 558)	0.06100024622136605
  (0, 700)	0.06993405576760808
  (0, 9674)	0.0869647643020172
  (0, 23789)	0.08112409485122009
  (0, 9942)	0.08131128704508334
  (0, 239)	0.20140237317433074
  (0, 4714)	0.12769944629731544
  (0, 19924)	0.19673522967318463
  (0, 16887)	0.14632066391543302
  (0, 7767)	0.18365890160698312
  (0, 18109)	0.15086902103897895
  (0, 19859)	0.3855784321368156
  (0, 28350)	0.15823322710681617
  (0, 17885)	0.11174263584921847
  (0, 27275)	0.1488200543150119
  (0, 10280)	0.14145584824717466
  (0, 19479)	0.16466647900154974
  (0, 21339)	0.17007565760107493
  (0, 1007)	0.12549607244507047
  (0, 5191)	0.1686124926063266
  (0, 23432)	0.11284550751496117
  (0, 29867)	0.18635596417367425
  (0, 30259)	0.1538840804706521
  (0, 23391)	0.17162755203799976
  (0, 24604)	0.16233774535695725
  :	:
  (4802, 9508)	0.10801504085274487
  (4802, 21202)	0.09308155260664236
  (4802, 9093)	0.08119343238402535
  (4802, 23914)	0.08512179276884631
  (4802, 6080)	0.1117337732868638
  (4802, 1

## Getting the similarity scores using cosine similarity

In [19]:
# Calculating cosine similarity between vectors in the feature_vector matrix
similarity = cosine_similarity(feature_vector, feature_vector)

In [20]:
print(similarity)

[[1.         0.02690871 0.02006298 ... 0.00422183 0.01891198 0.        ]
 [0.02690871 1.         0.01284585 ... 0.01876714 0.00946375 0.        ]
 [0.02006298 0.01284585 1.         ... 0.00887991 0.03856695 0.        ]
 ...
 [0.00422183 0.01876714 0.00887991 ... 1.         0.01336575 0.0223473 ]
 [0.01891198 0.00946375 0.03856695 ... 0.01336575 1.         0.00893746]
 [0.         0.         0.         ... 0.0223473  0.00893746 1.        ]]


In [21]:
print(similarity.shape)

(4803, 4803)


## Creating a Movie Recommendation System 

In [22]:
import difflib

# user input
movie_name = input('Enter your favourite movie name: ')
list_of_all_titles = movies_list['title'].tolist()

# finding colse match of the movie using difflib 
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

# case for no matches
if len(find_close_match) == 0:
    print("Sorry, no close matches found for the movie you entered.")
# If movie name matches
else:
    close_match = find_close_match[0]
    print(f"Did you mean: {close_match}?")

    # get_recommendations function
    def get_recommendations(title, similarity=similarity):
        # Finding the index of the movie with the title
        idx = movies_list[movies_list['title'] == title].index[0]
        
        # Getting the similarity row for the selected index 
        similarity_score = list(enumerate(similarity[idx]))
        
        # Sorting the movies based on the similarity scores
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        
        # Getting the scores of the 10 most similar movies
        similarity_score = similarity_score[1:11]
        
        # Getting the movie indices
        movie_indices = [i[0] for i in similarity_score]
        
        return movies_list['title'].iloc[movie_indices]

print('Movies suggested for you:\n')
recommendations = get_recommendations(close_match)
print(recommendations)


Enter your favourite movie name:  rush


Did you mean: Rush?
Movies suggested for you:

225                               Speed Racer
560                                    Driven
3100                          The Brown Bunny
4393                           Speedway Junky
405     The Fast and the Furious: Tokyo Drift
3072                   Atlas Shrugged Part II
1824                     The Face of an Angel
1652                     Dragonball Evolution
1820                         The Fifth Estate
3438                                  Control
Name: title, dtype: object
